<a href="https://colab.research.google.com/github/AnastasiaBrinati/Progetto-ML-23-24/blob/main/task3_centralizzato.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#TASK 3

#### import

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.optimizers import Adam
!pip install -q -U keras-tuner
import keras_tuner as kt
from sklearn.metrics import precision_score, recall_score, f1_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.1 MB/s eta 0:00:00


#### get the data

In [2]:
# Inizializza il seed per NumPy per riproducibilità
np.random.seed(42)

# Inizializza il seed per TensorFlow per riproducibilità
tf.random.set_seed(42)
# Load the MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Preprocessing
# Reshape and scale the input features
X_train = X_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
X_test = X_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0

# One-hot encode the target labels
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)


11490434/11490434 [==============================] - 0s 0us/step


 #### Tuning

In [ ]:
# Define the model builder function for hyperparameter tuning
def model_builder(hp):
    model = keras.Sequential()

    # Tune the number of convolutional layers
    for i in range(hp.Int('num_conv_layers', 1, 3)):
        model.add(layers.Conv2D(
            filters=hp.Int(f'filters_{i}', min_value=16, max_value=128, step=16),
            kernel_size=hp.Choice(f'kernel_size_{i}', values=[3, 5]),
            activation='relu',
            padding='same'
        ))
        model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(layers.Flatten())

    # Tune the number of units and activation function for each Dense layer
    for i in range(hp.Int('num_dense_layers', 1, 3)):
        model.add(layers.Dense(
            units=hp.Int(f'units_{i}', min_value=16, max_value=128, step=16),
            activation=hp.Choice(f'activation_{i}', values=['PReLU','LeakyReLU','relu', 'tanh', 'sigmoid'])
        ))

    # Output layer
    model.add(layers.Dense(10, activation='softmax'))

    # Tune the learning rate for the optimizer
    # hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    hp_learning_rate = 1e-2

    model.compile(optimizer=Adam(learning_rate=hp_learning_rate),
                  loss='categorical_crossentropy',
                  metrics=["accuracy", "categorical_accuracy", "top_k_categorical_accuracy"])

    return model

# Initialize the tuner
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='mnist_tuning')

# Perform hyperparameter search
tuner.search(X_train, y_train, epochs=2, validation_split=0.2)

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Build the model with the optimal hyperparameters
tuned_model = tuner.hypermodel.build(best_hps)


In [ ]:
best_hps.values

{'num_conv_layers': 2,
 'filters_0': 128,
 'kernel_size_0': 5,
 'num_dense_layers': 2,
 'units_0': 64,
 'activation_0': 'PReLU',
 'learning_rate': 0.001,
 'filters_1': 48,
 'kernel_size_1': 5,
 'filters_2': 112,
 'kernel_size_2': 3,
 'units_1': 80,
 'activation_1': 'relu',
 'units_2': 32,
 'activation_2': 'PReLU',
 'tuner/epochs': 10,
 'tuner/initial_epoch': 4,
 'tuner/bracket': 1,
 'tuner/round': 1,
 'tuner/trial_id': '0018'}

#### Model

In [54]:
# Build the actual model
def get_model():

  model = keras.Sequential()

  # (conv2 + pooling)*2
  model.add(layers.Conv2D(input_shape=(28, 28, 1), filters=1, kernel_size=5, activation='relu',padding='same'))

  model.add(layers.MaxPooling2D(pool_size=(2, 2)))

  model.add(layers.Conv2D(filters=48,kernel_size=5,activation='relu',padding='same'))

  model.add(layers.MaxPooling2D(pool_size=(2, 2)))

  # flatten layer
  model.add(layers.Flatten())

  # dense layers
  model.add(layers.Dense(units=64, activation="PReLU"))
  model.add(layers.Dense(units=80, activation="ReLU"))
  model.add(layers.Dense(units=32, activation="PReLU"))
  # output layer
  model.add(layers.Dense(10, activation='softmax'))

  model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=["accuracy", "categorical_accuracy", "top_k_categorical_accuracy"])
  return model

In [55]:
model = get_model()
model.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_25 (Conv2D)          (None, 28, 28, 1)         26        
                                                                 
 max_pooling2d_22 (MaxPooli  (None, 14, 14, 1)         0         
 ng2D)                                                           
                                                                 
 conv2d_26 (Conv2D)          (None, 14, 14, 48)        1248      
                                                                 
 max_pooling2d_23 (MaxPooli  (None, 7, 7, 48)          0         
 ng2D)                                                           
                                                                 
 flatten_10 (Flatten)        (None, 2352)              0         
                                                                 
 dense_39 (Dense)            (None, 64)              

#### history

In [62]:
# Train the model with the optimal hyperparameters
history = model.fit(X_train, y_train, epochs=10, validation_split=0.2, verbose=0)

# Evaluate the model on test data
test_loss, test_accuracy, test_categorical_accuracy, test_top_k_categorical_accuracy = model.evaluate(X_test, y_test)

# Predictions on the validation set
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Calculate precision, recall, and F1-score
test_precision = precision_score(np.argmax(y_test, axis=1), y_pred_classes, average='weighted')
test_recall = recall_score(np.argmax(y_test, axis=1), y_pred_classes, average='weighted')
test_f1_score = f1_score(np.argmax(y_test, axis=1), y_pred_classes, average='weighted')

# Display metrics
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)
print("Test Categorical Accuracy:", test_categorical_accuracy)
print("Test Top-K Categorical Accuracy:", test_top_k_categorical_accuracy)
print("Test Precision:", test_precision)
print("Test Recall:", test_recall)
print("Test F1 Score:", test_f1_score)

313/313 [==============================] - 3s 9ms/step
Test Loss: 0.05647412687540054
Test Accuracy: 0.9884999990463257
Test Categorical Accuracy: 0.9884999990463257
Test Top-K Categorical Accuracy: 1.0
Test Precision: 0.9885204587249998
Test Recall: 0.9885
Test F1 Score: 0.988500365550457


In [63]:
import pandas as pd

# Converti la cronologia in un DataFrame
history_df = pd.DataFrame(history.history)

# Stampa la tabella delle metriche per ogni epoca
print(history_df.to_string())


       loss  accuracy  categorical_accuracy  top_k_categorical_accuracy  val_loss  val_accuracy  val_categorical_accuracy  val_top_k_categorical_accuracy
0  0.006573  0.997979              0.997979                         1.0  0.074014      0.985750                  0.985750                        0.999667
1  0.009880  0.996938              0.996938                         1.0  0.080995      0.984750                  0.984750                        0.999500
2  0.005082  0.998417              0.998417                         1.0  0.095862      0.983250                  0.983250                        0.999667
3  0.007821  0.997604              0.997604                         1.0  0.078904      0.983333                  0.983333                        0.999500
4  0.006833  0.997896              0.997896                         1.0  0.078506      0.986083                  0.986083                        0.999333
5  0.007958  0.997604              0.997604                         1.0  0.0